In [1]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Dense,LSTM, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,consensus_score, accuracy_score

In [2]:
df = pd.read_csv('fake_news/train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df = df.dropna()

In [5]:
X = df.drop('label',axis=1)

In [6]:
y = df['label']

In [7]:
X.shape

(18285, 4)

In [8]:
y.shape

(18285,)

In [9]:
message = X.copy()
message.reset_index(inplace=True)

In [10]:
###ONEHOT ENCODING

In [11]:
message['title'][6]

'Benoît Hamon Wins French Socialist Party’s Presidential Nomination - The New York Times'

In [12]:
corpus = []
ps = PorterStemmer()
for i in tqdm(range(0,len(message))):
    review = re.sub('[^a-zA-Z]', ' ',message['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18285/18285 [00:35<00:00, 512.68it/s]


In [13]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [14]:
voc_size = 10000

In [15]:
onehot_rep = [one_hot(words,voc_size) for words in corpus]

In [16]:
onehot_rep

[[6820, 6994, 1468, 2641, 6535, 9386, 7964, 4009, 9002, 505],
 [971, 3869, 8941, 1747, 1611, 3178, 3981],
 [6603, 7599, 4256, 5724],
 [5231, 8516, 991, 3389, 557, 9487],
 [8682, 1611, 8898, 7401, 1116, 1682, 1611, 7980, 315, 9511],
 [4214,
  6476,
  2429,
  440,
  7937,
  6213,
  5118,
  3551,
  2204,
  4323,
  7489,
  280,
  1685,
  6877,
  3981],
 [2465, 2048, 4574, 1255, 9155, 8889, 4003, 4297, 439, 4436, 6074],
 [3662, 5249, 8908, 27, 5517, 4963, 6213, 6810, 439, 4436, 6074],
 [4068, 5260, 9939, 4046, 7510, 7880, 308, 4049, 6213, 831],
 [3645, 2825, 1395, 1646, 9385, 2394, 7298, 683],
 [8269, 4529, 7292, 4774, 6314, 4336, 7988, 4637, 2180, 122, 1453],
 [3389, 7692, 6535, 7880, 6213, 5517],
 [8416, 175, 4119, 6075, 129, 27, 1400, 4649, 4106],
 [7294, 830, 5325, 1347, 8900, 910, 8876, 439, 4436, 6074],
 [3368, 1812, 7882, 2007, 684, 439, 4436, 6074],
 [3522, 9265, 7536, 6126, 2138, 4544, 7689, 4999, 9872, 1011],
 [5579, 3082, 3869],
 [6175, 3885, 5636, 967, 6213, 1864, 8505, 3981],
 

In [17]:
###Embeding Representation

In [18]:
sent_len = 20
embeded_doc = pad_sequences(onehot_rep,padding='pre',maxlen = sent_len)
embeded_doc[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 6820,
       6994, 1468, 2641, 6535, 9386, 7964, 4009, 9002,  505], dtype=int32)

In [19]:
##creating model

In [20]:
vector_size = 40   #dimension
model = Sequential()
model.add(Embedding(voc_size,vector_size, input_length = sent_len))
model.add(LSTM(10))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy',optimizer= 'adam', metrics=['accuracy'])

2021-08-19 20:47:43.779937: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-19 20:47:43.794687: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fa037e330f0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-08-19 20:47:43.794705: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            400000    
_________________________________________________________________
lstm (LSTM)                  (None, 10)                2040      
_________________________________________________________________
dense (Dense)                (None, 1)                 11        
Total params: 402,051
Trainable params: 402,051
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.predict(embeded_doc)

array([[0.50301117],
       [0.49779657],
       [0.49565396],
       ...,
       [0.49396774],
       [0.49365664],
       [0.496586  ]], dtype=float32)

In [23]:
len(embeded_doc),y.shape

(18285, (18285,))

In [24]:
X_final = np.array(embeded_doc)
Y_final = np.array(y)

In [25]:
X_final.shape,Y_final.shape

((18285, 20), (18285,))

In [26]:
x_train, x_test, y_train, y_test = train_test_split(X_final, Y_final,test_size= 0.25, random_state=0)

In [27]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size = 64)

Epoch 1/10
215/215 [==============================] - 2s 9ms/step - loss: 0.4050 - accuracy: 0.8313 - val_loss: 0.2357 - val_accuracy: 0.9121
Epoch 2/10
215/215 [==============================] - 2s 8ms/step - loss: 0.1558 - accuracy: 0.9467 - val_loss: 0.1947 - val_accuracy: 0.9208
Epoch 3/10
215/215 [==============================] - 2s 9ms/step - loss: 0.0959 - accuracy: 0.9705 - val_loss: 0.2086 - val_accuracy: 0.9213
Epoch 4/10
215/215 [==============================] - 2s 7ms/step - loss: 0.0658 - accuracy: 0.9815 - val_loss: 0.2272 - val_accuracy: 0.9171
Epoch 5/10
215/215 [==============================] - 2s 7ms/step - loss: 0.0484 - accuracy: 0.9869 - val_loss: 0.2525 - val_accuracy: 0.9154
Epoch 6/10
215/215 [==============================] - 2s 7ms/step - loss: 0.0370 - accuracy: 0.9915 - val_loss: 0.2750 - val_accuracy: 0.9160
Epoch 7/10
215/215 [==============================] - 2s 8ms/step - loss: 0.0286 - accuracy: 0.9927 - val_loss: 0.2944 - val_accuracy: 0.9147
Epoch 

In [28]:
##Adding Dropout

In [29]:
#creating model 
model = Sequential()
model.add(Embedding(voc_size,vector_size, input_length = sent_len))
model.add(Dropout(0.3))
model.add(LSTM(10))
model.add(Dropout(0.3))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy',optimizer= 'adam', metrics=['accuracy'])

In [30]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size = 64)

Epoch 1/10
215/215 [==============================] - 2s 10ms/step - loss: 0.4096 - accuracy: 0.8177 - val_loss: 0.2238 - val_accuracy: 0.9101
Epoch 2/10
215/215 [==============================] - 2s 8ms/step - loss: 0.1716 - accuracy: 0.9383 - val_loss: 0.1860 - val_accuracy: 0.9199
Epoch 3/10
215/215 [==============================] - 2s 7ms/step - loss: 0.1104 - accuracy: 0.9637 - val_loss: 0.1916 - val_accuracy: 0.9206
Epoch 4/10
215/215 [==============================] - 2s 7ms/step - loss: 0.0783 - accuracy: 0.9762 - val_loss: 0.2126 - val_accuracy: 0.9182
Epoch 5/10
215/215 [==============================] - 2s 8ms/step - loss: 0.0597 - accuracy: 0.9835 - val_loss: 0.2314 - val_accuracy: 0.9186
Epoch 6/10
215/215 [==============================] - 2s 8ms/step - loss: 0.0481 - accuracy: 0.9883 - val_loss: 0.2552 - val_accuracy: 0.9217
Epoch 7/10
215/215 [==============================] - 2s 8ms/step - loss: 0.0384 - accuracy: 0.9902 - val_loss: 0.2771 - val_accuracy: 0.9164
Epoch

In [31]:
#Performance Metrics And Accuracy


In [32]:
y_pred = model.predict_classes(x_test)


Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [33]:
confusion_matrix(y_test,y_pred)

array([[2406,  177],
       [ 227, 1762]])

In [34]:
accuracy_score(y_test,y_pred)

0.9116360454943132